# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [325]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [326]:
# 1. FeatureExtractor
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Извлечение часа и дня недели из timestamp
        X = X.copy()
        X['hour'] = pd.to_datetime(X['timestamp']).dt.hour
        X['dayofweek'] = pd.to_datetime(X['timestamp']).dt.weekday
        X.drop(columns=['timestamp'], inplace=True)
        return X


In [327]:
# 2. MyOneHotEncoder
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_column):
        self.target_column = target_column
        self.encoder = OneHotEncoder(sparse_output=False, drop='first')

    def fit(self, X, y=None):
        # Определяем категориальные признаки (исключая целевую переменную)
        self.categorical_features = X.select_dtypes(include=['object', 'category']).columns
        if self.target_column in self.categorical_features:
            self.categorical_features = self.categorical_features.drop(self.target_column)
        
        # Обучаем OneHotEncoder на категориальных признаках
        if len(self.categorical_features) > 0:
            self.encoder.fit(X[self.categorical_features])
        return self

    def transform(self, X):
        X = X.copy()
        if len(self.categorical_features) > 0:
            # Применяем OneHotEncoder к категориальным признакам
            encoded_features = self.encoder.transform(X[self.categorical_features])
            encoded_df = pd.DataFrame(encoded_features, columns=self.encoder.get_feature_names_out(self.categorical_features))
            
            # Удаляем исходные категориальные признаки и добавляем закодированные
            X.drop(columns=self.categorical_features, inplace=True)
            X = pd.concat([X, encoded_df], axis=1)
        
        # Возвращаем признаки и целевую переменную
        y = X[self.target_column]
        X.drop(columns=[self.target_column], inplace=True)
        return X, y

In [328]:
# 3. TrainValidationTest
class TrainValidationTest(BaseEstimator, TransformerMixin):
    def __init__(self, test_size=0.2, random_state=21):
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y=None):
        return self

    def transform(self, X, y):
        # Разделение на обучающую и тестовую выборки
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        # Разделение обучающей выборки на обучающую и валидационную
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_train, y_train, test_size=self.test_size, random_state=self.random_state, stratify=y_train
        )
        
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [329]:
class ModelSelection:
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
        self.results = []

    def choose(self, X_train, y_train, X_valid, y_valid):
        best_score = 0
        best_model_name = None

        for idx, grid in enumerate(self.grids):
            model_name = self.grid_dict[idx]
            print(f"Estimator: {model_name}")

            pbar = tqdm(total=1, desc="Fitting model")
            pbar.update()
            pbar.close()

            # Обучение GridSearchCV
            grid.fit(X_train, y_train)

            # Прогнозирование на валидационном наборе данных
            y_valid_pred = grid.predict(X_valid)
            valid_score = accuracy_score(y_valid, y_valid_pred)

            # Сохранение результатов
            self.results.append({
                'model': model_name,
                'params': grid.best_params_,
                'valid_score': valid_score
            })

            # Вывод информации
            print(f"Best params: {grid.best_params_}")
            print(f"Best training accuracy: {grid.best_score_:.3f}")
            print(f"Validation set accuracy score for best params: {valid_score:.3f}\n")

            # Обновление лучшей модели
            if valid_score > best_score:
                best_score = valid_score
                best_model_name = model_name

        print(f"Classifier with best validation set accuracy: {best_model_name}")
        return best_model_name

    def best_results(self):
        return pd.DataFrame(self.results)

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [330]:
class Finalize:
    def __init__(self, estimator):
        self.estimator = estimator

    def final_score(self, X_train, y_train, X_test, y_test):
        # Обучаем модель на обучающем наборе
        self.estimator.fit(X_train, y_train)
        # Получаем прогнозы и считаем точность
        accuracy = self.estimator.score(X_test, y_test)
        print(f"Accuracy of the final model is {accuracy:.6f}")
        return accuracy

    def save_model(self, path):
        dump(self.estimator, path)
        print(f"Model successfully saved to {path}")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [331]:
# 1. Загрузка данных
df = pd.read_csv("../data/checker_submits.csv")

In [332]:
# 2. Создание preprocessing pipeline
preprocessing = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('onehot_encoder', MyOneHotEncoder('dayofweek'))
])


In [333]:
# 3. Применение pipeline
data = preprocessing.fit_transform(df)

In [334]:
# 4. Разделение данных
splitter = TrainValidationTest(test_size=0.2, random_state=21)
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.transform(*data)

In [335]:
# 5. Выбор лучшей модели
# Параметры для GridSearchCV
svm_params = [{'kernel': ['linear', 'rbf', 'sigmoid'], 'C': [0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight': ['balanced', None], 'random_state': [21], 'probability': [True]}]
tree_params = [{'max_depth': range(1, 50), 'class_weight': ['balanced', None], 'criterion': ['gini', 'entropy'], 'random_state': [21]}]
rf_params = [{'n_estimators': [50, 100, 200], 'max_depth': range(1, 50), 'class_weight': ['balanced', None], 'criterion': ['gini', 'entropy'], 'random_state': [21]}]

# Инициализация GridSearchCV
gs_svm = GridSearchCV(estimator=SVC(), param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_tree = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=tree_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)

In [336]:
# Список GridSearchCV и словарь с именами моделей
grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

In [337]:
# Инициализация ModelSelection
model_selection = ModelSelection(grids, grid_dict)

In [338]:
# Выбор лучшей модели
best_model_name = model_selection.choose(X_train, y_train, X_valid, y_valid)

Estimator: SVM


Fitting model:   0%|          | 0/1 [00:00<?, ?it/s]

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.768
Validation set accuracy score for best params: 0.885

Estimator: Decision Tree


Fitting model:   0%|          | 0/1 [00:00<?, ?it/s]

Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.844

Estimator: Random Forest


Fitting model:   0%|          | 0/1 [00:00<?, ?it/s]

Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 31, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.859
Validation set accuracy score for best params: 0.893

Classifier with best validation set accuracy: Random Forest


In [339]:
# Получение лучших результатов
best_results_df = model_selection.best_results()
best_results_df

,model,params,valid_score
0,SVM,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.885185
1,Decision Tree,"{'class_weight': 'balanced', 'criterion': 'gin...",0.844444
2,Random Forest,"{'class_weight': None, 'criterion': 'entropy',...",0.892593


In [340]:
# 6. Финализация модели
model_key = next(key for key, value in grid_dict.items() if value == best_model_name)
best_model = grids[model_key]

final = Finalize(best_model)

accuracy = final.final_score(X_train, y_train, X_test, y_test)

Accuracy of the final model is 0.914201


In [341]:
# Сохранение модели
model_filename = f"../model/{best_model_name}_{accuracy:.4f}.joblib"
final.save_model(model_filename)

Model successfully saved to ../model/Random Forest_0.9142.joblib
